In [1]:
import pandas as pd
data = {
    'person_age': [22, 21, 25, 23, 24, 21, 21, 25],
    'person_income': [59000, 9600, 9600, 65500, 54400, 9900, 11389, 306000],
    'person_home_ownership': ['RENT', 'OWN', 'MORTGAGE', 'RENT', 'RENT', 'OWN', 'OTHER', 'RENT'],
    'person_emp_length': [123, 5, 1, 4, 8, 2, 5, 2],
    'loan_intent': ['PERSONAL', 'EDUCATION', 'MEDICAL', 'MEDICAL', 'MEDICAL', 'VENTURE', 'EDUCATION', 'DEBTCONSOLIDATION'],
    'loan_grade': ['D', 'B', 'G', 'C', 'C', 'A', 'E', 'F'],
    'loan_amnt': [35000, 1000, 5500, 35000, 35000, 2500, 4000, 24250],
    'loan_int_rate': [16.02, 11.14, 12.87, 15.23, 14.27, 7.14, 12.84, 13.85],
    'loan_percent_income': [0.59, 0.1, 0.57, 0.53, 0.55, 0.25, 0.35, 0.08],
    'cb_person_default_on_file': ['Y', 'N', 'N', 'N', 'Y', 'N', 'Y', 'N'],
    'cb_person_cred_hist_length': [3, 2, 3, 2, 4, 2, 2, 3]
}

test_data = pd.DataFrame(data)
test_data.to_csv('new_data.csv',header=True)

In [2]:
test_data.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123,PERSONAL,D,35000,16.02,0.59,Y,3
1,21,9600,OWN,5,EDUCATION,B,1000,11.14,0.10,N,2
2,25,9600,MORTGAGE,1,MEDICAL,G,5500,12.87,0.57,N,3
3,23,65500,RENT,4,MEDICAL,C,35000,15.23,0.53,N,2
4,24,54400,RENT,8,MEDICAL,C,35000,14.27,0.55,Y,4


In [ ]:
from pyspark.ml import Transformer
from pyspark.ml.param.shared import Param, Params
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
from pyspark.ml.pipeline import Pipeline
class OneHotEncoderCustom(Transformer, DefaultParamsReadable, DefaultParamsWritable):
    def __init__(self, encoding_columns=None):
        super(OneHotEncoderCustom, self).__init__()
        self.encoding_columns = Param(self, "encoding_columns", "columns to be one-hot encoded")

        if encoding_columns is not None:
            self._set(encoding_columns=encoding_columns)

    def _transform(self, df: DataFrame) -> DataFrame:
        encoding_columns = self.getOrDefault(self.encoding_columns)

        for column in encoding_columns:
            distinct_values = df.select(column).distinct().rdd.map(lambda x: x[0]).collect()

            # Create one-hot encoded columns
            for value in distinct_values:
                df = df.withColumn(f"{column}_{value}", F.when(F.col(column) == value, 1).otherwise(0))

            # Drop the original column
            df = df.drop(column)

        return df

    def getEncodingColumns(self):
        return self.getOrDefault(self.encoding_columns)

# Columns to encode
encoding_columns = ['cb_person_default_on_file', 'loan_grade', 'person_home_ownership', 'loan_intent', 'income_group', 'age_group', 'loan_amount_group']

one_hot_encoder = OneHotEncoderCustom(encoding_columns=encoding_columns)
pipeline = Pipeline(stages=[one_hot_encoder])

pipeline_model = pipeline.fit(training_data)

# Saving the pipeline model
pipeline_model.save("./encoding_pipeline")

# Loading the pipeline model

